# Building Logic with Circuits and Understanding Noise Levels

This tutorial aims at introducing you to building a some simple classical logic from the elementary gates used in quantum computing and using the results to show just how noisy the current quantum hardware still is. The aim is to appreciate the amount of work ahead for the industry before we get to fault tolerant quantum computers.

Concretely, we build a two qubit adder (a half adder) and run it to see what 1+1 is. 

We start by importing tools we will be using, but there is no need to understand them in more detail.

In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.ibmq import least_busy
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

To perform our calculation we will first need two qubits to be summed (the input).

As any qubit can have a value of 0 at minimum and 1 at maximum we can have anything between 0 and 2 as the result of our addition. This means we will also need two qubits for the output to be able to store values larger than 1.

This makes four qubits in total. 

Let's also create matching four classical bits, so that we can measure and display the result of all of our qubits in corresponding classical bits.

In [ ]:
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

Let's see what it looks like with a handy command!

In [ ]:
qc.draw()

Doesn't look like much yet. Note that the classical bits are denoted by just one double line and the number four written to denote the bits in total.

Next, we'll look at two quantum gates that we will be using and combining to create our adder. There is no unique one way to build such an adder and thus we could've used other kinds of gates as well.

Since our problem is classical, there isn't much we could do with single qubit gates. It turns out that we only need the CONTROLLED NOT -gate (CNOT or CX) and a similar gate with two control qubits. The effect of the CNOT is to flip the target qubit from 0 to 1 or from 1 to 0, if and only if, the control qubit is 1.

Let's add one to our circuit with qubit zero as the control and three as the target and draw the circuit again.

In [ ]:
qc.cx(q[0], q[3])
qc.draw()

With the first gate added the circuit starts to do something. We can think of the zero and one qubits being our input and the two and three qubits the output. We can also decide that the qubit two is the more significant qubit of the output and the qubit three the less significant one. 

Now our circuit outputs 01, if the first qubit of the input was a 1. This makes sense for an adder as we output 1 when we see a 1 in the input.

The second type of gate we mentioned and we are going to need is a CONTROLLED NOT gate with two control qubits. It flips the value of the target qubit only if both of the control qubits are ones. This gate has a symphathetic name: Toffoli. And it can be referred also as CCNOT or CCX.

Let's add one Toffoli gate with control qubits zero and one and the target qubit two and look what we have.

In [ ]:
qc.ccx(q[0],q[1],q[2])
qc.draw()

We are actually really close to our result. Let's run a few calculations with our current circuit to figure out what we are still missing.

First we need to connect to a backend that runs the actual calculation. We are using a simulator at this point, because we wouldn't benefit at all from a real device.

In [ ]:
backend = least_busy(IBMQ.get_provider().backends(simulator=True))

All the qubits start out as zero, so we do the first calculation of adding 0+0 and expect to see 00 as the result in all 32 tries.

In [ ]:
qc.measure(q, c) #We need to measure the qubits to the classical bits to be able to read the results.
#Note: This adds measurements to the actual circuit.
job_exp = execute(qc, backend=backend, shots=32, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc)) #Let's draw the results for easier viewing.

Works like a charm.

Next we'll turn one of the inputs into a 1 with the NOT gate (also called X), but let's create the circuit from scratch to have this happen before our calculation starts and to get rid of the measurement gates we also just added.

In [ ]:
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

qc.x(q[0]) #The qubit zero is now a 1.

#Our actual circuit
qc.cx(q[0], q[3])
qc.ccx(q[0], q[1],q[2])
qc.draw()

Nice! And now calculating 1+0.

In [ ]:
qc.measure(q, c)
job_exp = execute(qc, backend=backend, shots=32, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc))

That worked as well. Now let's do 1+1 - we need to add a NOT to the other input as well.

In [ ]:
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

qc.x(q[0]) #The qubit zero is now a 1.
qc.x(q[1]) #The other input to 1.

#Our actual circuit
qc.cx(q[0], q[3])
qc.ccx(q[0], q[1],q[2])
qc.draw()

This time 1+1 should give us 10.

In [ ]:
qc.measure(q, c)
job_exp = execute(qc, backend=backend, shots=32, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc))

Instead we got 11, which would be 3 in decimal numbers. Why?

Take a moment and try to add a new gate to the circuit to fix the problem.

In [ ]:
#THIS CODE BLOCK IS FOR YOU TO ADD YOUR GATE
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

qc.x(q[0]) #The qubit zero is now a one.
qc.x(q[1]) #The other to one.

#Our actual circuit - ADD A GATE HERE 
qc.cx(q[0], q[3])
qc.ccx(q[0], q[1],q[2])

qc.draw()

In [ ]:
#THIS CODE BLOCK IS FOR YOU TO TEST YOUR ADDED GATE
qc.measure(q, c)
job_exp = execute(qc, backend=backend, shots=32, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc))

The solution is another CNOT that will flip qubit three back to 0, if both of the inputs are 1.

Here's the final adder with the inputs now set 0.

In [ ]:
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

qc.cx(q[0], q[3])
qc.cx(q[1], q[3])  
qc.ccx(q[0], q[1],q[2])

qc.draw()

We promised to also look at how much noise the actual quantum hardware has. With our simulations we have always gotten all the 32 tries right. Let's do a run of 1024 calculations, but with a quantum twist.

We will put a Hadamard gate to both of the inputs, which means that they will be both zero and one at the same time with equal probability.

Let's first see the circuit.

In [ ]:
q = QuantumRegister(4) #The four qubits
c = ClassicalRegister(4) #The matching four classical bits
qc = QuantumCircuit(q, c) #And they together make up our circuit

qc.h(q[0])
qc.h(q[1])  

qc.cx(q[0], q[3])
qc.cx(q[1], q[3])  
qc.ccx(q[0], q[1],q[2])

qc.draw()

With the 1024 tries, we should get 25% of the time 00, 25% of the time 10 and 50% of the time 01, since there are two ways to get to the last result - either qubit zero is 1 or qubit one is 1. And we should never arrive at 11.

Next we'll run the simulation to see how close we get. Remember that we are simulating randomness, so the distribution of the inputs won't be exactly as set out, but we should never get 11 as that would be calculation error and not randomness of the inputs.

In [ ]:
qc.measure(q, c)
job_exp = execute(qc, backend=backend, shots=1024, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc))

And finally we change to the real hardware for comparison and run the calculation again.

This might take a while as the backends can be quite busy. Remember what a piece of magic it is to really run the calculation on an actual quantum device!

In [ ]:
backend = least_busy(IBMQ.get_provider().backends(simulator=False))

#No need to add the measurements to the circuit as they are there already.
job_exp = execute(qc, backend=backend, shots=1024, max_credits=3)
plot_histogram(job_exp.result().get_counts(qc))

The results are all over the place, which already shows how much work we still have ahead of us.

Let's still calculate the share of cases that were calculate correctly.

In [ ]:
data = job_exp.result().get_counts(qc)
if len(data) == 16:
    maybe_correct = data["0000"] + data["1010"] + data["1001"] + data["0111"]
    incorrect = data["0001"] + data["0010"] + data["0011"] + data["1000"] + data["1011"] + data["0100"] + data["0101"] + data["0110"] + data["1100"] + data["1101"] + data["1110"] + data["1111"]
    print("Tries that either have 0 or an even number of errors: ", maybe_correct)
    print("Tries that have an odd number of errors: ", incorrect)
    print("Share of correct results: ", maybe_correct / (incorrect+maybe_correct) * 100, "%")